<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Four: Neural networks </h2>	


https://towardsdatascience.com/training-a-convolutional-neural-network-from-scratch-2235c2a25754
https://www.learnopencv.com/image-classification-using-convolutional-neural-networks-in-keras/
https://visualstudiomagazine.com/articles/2018/12/01/image-classification-keras.aspx
https://medium.com/nybles/create-your-first-image-recognition-classifier-using-cnn-keras-and-tensorflow-backend-6eaab98d14dd
https://towardsdatascience.com/image-recognition-with-keras-convolutional-neural-networks-e2af10a10114
https://github.com/keras-team/keras/issues/3109


In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from tqdm import tqdm

import seaborn as sns

import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import skimage
import sklearn
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LassoCV
from sklearn.tree import DecisionTreeClassifier


from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
#Dataset load and preprocessing SOURCE Rohan BAIR
def read_data():
  
  img_list, label_list = [], []

  file_path = os.path.join(os.getcwd(), 'drive/My Drive/D100_Final_Project/20_categories_training')
    
 
  folders = os.listdir(file_path)
  folders.sort()

  i = 0
  for file_name in folders: #for each subfolder
      if file_name.startswith('.'): #image name format ._
          continue
      print(file_name) #checkpoint

      for image_file in os.listdir(os.path.join(file_path, file_name)):
          if image_file.startswith('.'):
              continue
          image_file = os.path.join(file_path, file_name, image_file)
          image = cv2.imread(image_file)
          #resize each image to the shorter median length 300 
          image = cv2.resize(image, (300, 300), cv2.INTER_LINEAR)
          img_list.append(image)
          label_list.append(i)
      i += 1
  img_list = np.array(img_list, dtype=np.uint8)
  label_list= np.array( label_list, dtype=np.int32 )
  img_list = img_list.astype('float32')
  #Scale down, normalizing the data
  img_list = img_list / 255
  return img_list, label_list

data, label = read_data()
print(data.shape, label.shape)

airplanes
bear
blimp
comet
crab
dog
dolphin
giraffe
goat
gorilla
kangaroo
killer-whale
leopards
llama
penguin
porcupine
teddy-bear
triceratops
unicorn
zebra
(1501, 300, 300, 3) (1501,)


In [51]:
def read_val_data():
  img_list= []
  file_path = os.path.join(os.getcwd(), 'drive/My Drive/D100_Final_Project/20_Validation')
  files = os.listdir(file_path)
  files.sort()
   
  for image_file in files: #for each subfolder     
      print(image_file) #checkpoint
      image_file = os.path.join(file_path, image_file)
      image = cv2.imread(image_file)
      #resize each image to the shorter median length 300 
      image = cv2.resize(image, (300,300), cv2.INTER_LINEAR)
      img_list.append(image)
  img_list = np.array(img_list, dtype=np.uint8)
  img_list = img_list.astype('float32')
  #Scale down
  img_list = img_list / 255
  return img_list

val_images= read_val_data()
print(val_images.shape)

validation_pic (1).jpg
validation_pic (10).jpg
validation_pic (100).jpg
validation_pic (101).jpg
validation_pic (102).jpg
validation_pic (103).jpg
validation_pic (104).jpg
validation_pic (105).jpg
validation_pic (106).jpg
validation_pic (107).jpg
validation_pic (108).jpg
validation_pic (109).jpg
validation_pic (11).jpg
validation_pic (110).jpg
validation_pic (111).jpg
validation_pic (112).jpg
validation_pic (113).jpg
validation_pic (114).jpg
validation_pic (115).jpg
validation_pic (116).jpg
validation_pic (117).jpg
validation_pic (118).jpg
validation_pic (119).jpg
validation_pic (12).jpg
validation_pic (120).jpg
validation_pic (121).jpg
validation_pic (122).jpg
validation_pic (123).jpg
validation_pic (124).jpg
validation_pic (125).jpg
validation_pic (126).jpg
validation_pic (127).jpg
validation_pic (128).jpg
validation_pic (129).jpg
validation_pic (13).jpg
validation_pic (130).jpg
validation_pic (131).jpg
validation_pic (132).jpg
validation_pic (133).jpg
validation_pic (134).jpg
valida

In [0]:
#Shuffle indices to get random ordering of images and classes
indices = np.arange(data.shape[0] )
np.random.shuffle(indices)
data, label = data[indices], label[indices]
#Test Train Split
ratio = 0.8
split = np.int(data.shape[0]* ratio)
x_train, y_train = data[:split], label[:split]
x_val, y_val =data[split:], label[split:]

In [53]:
print(y_train)

[ 7 11 17 ... 11  2  6]


Build a neural network classifier using an architecture of your choosing. This application
of deep learning can be done in PyTorch, TensorFlow, or a framework of your choice. This is the
industry standard for image classification. Describe your network and assess its performance. To
receive extra credit, your neural network classifier must outperform your other methods.



In [54]:
data.shape

(1501, 300, 300, 3)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization, Convolution2D

In [0]:
model = Sequential() #conv - batchnorm - relu OR conv - relu - pool
model.add(Convolution2D(32, 3, 3, border_mode ="same", input_shape=(300, 300, 3)))
# model.add(BatchNormalization()) #GPU crash
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2),dim_ordering='th'))

model.add(Convolution2D(64, 2, 2, border_mode ="same"))
# model.add(BatchNormalization()) #GPU crash
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2),dim_ordering='th'))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(20, activation='softmax'))

In [57]:
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])
model.summary()

hist = model.fit(x_train, y_train, batch_size = 32, epochs = 10)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 300, 300, 32)      896       
_________________________________________________________________
activation_28 (Activation)   (None, 300, 300, 32)      0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 300, 150, 16)      0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 300, 150, 64)      4160      
_________________________________________________________________
activation_29 (Activation)   (None, 300, 150, 64)      0         
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 300, 75, 32)       0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 720000)          

ResourceExhaustedError: ignored